<a href="https://colab.research.google.com/github/ashikshafi08/Learning_Tensorflow/blob/main/Exercise/%F0%9F%9B%A0_09_Milestone_Project_2_SkimLit_%F0%9F%93%84%F0%9F%94%A5_Exercise_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠 09. Milestone Project 2: SkimLit 📄🔥 Exercise Solutions.

> **Note** The orders of the exercise is mixed. 


1. Checkout the [Keras guide on using pretrained GloVe embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/). Can you get this working with one of our models?
  - Hint: You'll want to incorporate it with a custom token [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.
  - It's up to you whether or not you fine-tune the GloVe embeddings or leave them frozen.

2. Try replacing the TensorFlow Hub Universal Sentence Encoder pretrained embedding for the [TensorFlow Hub BERT PubMed expert](https://tfhub.dev/google/experts/bert/pubmed/2) (a language model pretrained on PubMed texts) pretrained embedding. Does this effect results?

  - Note: Using the BERT PubMed expert pretrained embedding requires an extra preprocessing step for sequences (as detailed in the [TensorFlow Hub guide](https://tfhub.dev/google/experts/bert/pubmed/2)).
  - Does the BERT model beat the results mentioned in this paper? https://arxiv.org/pdf/1710.06071.pdf. 

3. What happens if you were to merge our `line_number` and `total_lines` features for each sequence? For example, created a `X_of_Y` feature instead? Does this effect model performance?
  - Another example: `line_number=1` and total_lines=11 turns into `line_of_X=1_of_11`.

4. Train `model_5` on all of the data in the training dataset for as many epochs until it stops improving. Since this might take a while, you might want to use:
  - `tf.keras.callbacks.ModelCheckpoint` to save the model's best weights only.
  - `tf.keras.callbacks.EarlyStopping` to stop the model from training once the validation loss has stopped improving for ~3 epochs.

5. Write a function (or series of functions) to take a sample abstract string, preprocess it (in the same way our model has been trained), make a prediction on each sequence in the abstract and return the abstract in the format:
```
PREDICTED_LABEL: SEQUENCE
PREDICTED_LABEL: SEQUENCE
PREDICTED_LABEL: SEQUENCE
PREDICTED_LABEL: SEQUENCE
```
You can find your own unstrcutured RCT abstract from PubMed or try this one from: [Baclofen promotes alcohol abstinence in alcohol dependent cirrhotic patients with hepatitis C virus (HCV) infection.](https://pubmed.ncbi.nlm.nih.gov/22244707/)

## Downloading the data and preprocessing it. 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers 

In [2]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (33/33), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [3]:
# Start by using the 20k dataset
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [4]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 'pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

In [5]:
# Create function to read the lines of a document
def get_lines(filename):
  with open(filename, "r") as f:
    return f.readlines()

In [6]:
# Creating a preprocessing function that returns a dictionary
def preprocess_text_with_line_numbers(filename):
  """Returns a list of dictionaries of abstract line data.

  Takes in filename, reads its contents and sorts through each line,
  extracting things like the target label, the text of the sentence,
  how many sentences are in the current abstract and what sentence number
  the target line is.

  Args:
      filename: a string of the target text file to read and extract line data
      from.

  Returns:
      A list of dictionaries each containing a line from an abstract,
      the lines label, the lines position in the abstract and the total number
      of lines in the abstract where the line is from. For example:

      [{"target": 'CONCLUSION',
        "text": The study couldn't have gone better, turns out people are kinder than you think",
        "line_number": 8,
        "total_lines": 8}]
  """
  input_lines = get_lines(filename) # get all lines from filename
  abstract_lines = "" # create an empty abstract
  abstract_samples = [] # create an empty list of abstracts
  
  # Loop through each line in target file
  for line in input_lines:
    if line.startswith("###"): # check to see if line is an ID line
      abstract_id = line
      abstract_lines = "" # reset abstract string
    elif line.isspace(): # check to see if line is a new line
      abstract_line_split = abstract_lines.splitlines() # split abstract into separate lines

      # Iterate through each line in abstract and count them at the same time
      for abstract_line_number, abstract_line in enumerate(abstract_line_split):
        line_data = {} # create empty dict to store data from line
        target_text_split = abstract_line.split("\t") # split target label from text
        line_data["target"] = target_text_split[0] # get target label
        line_data["text"] = target_text_split[1].lower() # get target text and lower it
        line_data["line_number"] = abstract_line_number # what number line does the line appear in the abstract?
        line_data["total_lines"] = len(abstract_line_split) - 1 # how many total lines are in the abstract? (start from 0)
        abstract_samples.append(line_data) # add line data to abstract samples list
    
    else: # if the above conditions aren't fulfilled, the line contains a labelled sentence
      abstract_lines += line
  
  return abstract_samples

In [7]:
# Get data from file and preprocess it
%%time
train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt") 
test_samples = preprocess_text_with_line_numbers(data_dir + "test.txt")

len(train_samples), len(val_samples), len(test_samples)

CPU times: user 396 ms, sys: 88.1 ms, total: 484 ms
Wall time: 484 ms


In [8]:
# Loading our data into a dataframe
import pandas as pd
train_df = pd.DataFrame(train_samples)
val_df = pd.DataFrame(val_samples)
test_df = pd.DataFrame(test_samples)
train_df.head(14)

,target,text,line_number,total_lines
0,OBJECTIVE,to investigate the efficacy of @ weeks of dail...,0,11
1,METHODS,a total of @ patients with primary knee oa wer...,1,11
2,METHODS,outcome measures included pain reduction and i...,2,11
3,METHODS,pain was assessed using the visual analog pain...,3,11
4,METHODS,secondary outcome measures included the wester...,4,11
5,METHODS,"serum levels of interleukin @ ( il-@ ) , il-@ ...",5,11
6,RESULTS,there was a clinically relevant reduction in t...,6,11
7,RESULTS,the mean difference between treatment arms ( @...,7,11
8,RESULTS,"further , there was a clinically relevant redu...",8,11
9,RESULTS,these differences remained significant at @ we...,9,11


In [9]:
# Convert abstract text lines into lists 
train_sentences = train_df["text"].tolist()
val_sentences = val_df["text"].tolist()
test_sentences = test_df["text"].tolist()
len(train_sentences), len(val_sentences), len(test_sentences)

(180040, 30212, 30135)

In [10]:
# One hot encoding the labels 
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_df["target"].to_numpy().reshape(-1, 1))
test_labels_one_hot = one_hot_encoder.transform(test_df["target"].to_numpy().reshape(-1, 1))

# Check what training labels look like
train_labels_one_hot

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [11]:
# Extract labels and encoder them into integers 
from sklearn.preprocessing import LabelEncoder 

label_encoder = LabelEncoder() 

train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(val_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test_df["target"].to_numpy())

# Check what training labels look like
train_labels_encoded

array([3, 2, 2, ..., 4, 1, 1])

In [12]:
# Get class names and number of classes from LabelEncoder instance 
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes , class_names

(5, array(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'],
       dtype=object))


### 1. Checkout the [Keras guide on using pretrained GloVe embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/). Can you get this working with one of our models?

In [13]:
# Loading the pre-trained embeddings 
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-09-02 08:25:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-02 08:25:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-02 08:25:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [23]:
# Getting the path of the glove embedding (using 100D)
import numpy as np 
glove_path = 'glove.6B.200d.txt'

embedding_index = {}

# Making dict of vector representtion of the words (s --> [8, 48......])
with open(glove_path) as f:
  for line in f:
    
    # Getting the words and coef in a variable 
    word , coefs = line.split(maxsplit = 1)
    coefs = np.fromstring(coefs , 'f' , sep = ' ')
    
    # Adding the coefs to our embedding dict 
    embedding_index[word] = coefs

print(f'Found {len(embedding_index)} word vectors')

Found 400000 word vectors


Great we loaded in the data and the next step will be creating a corresponding embedding matrix. So we can fit our `embedding_index` to our Embedding layer. 


In [24]:
# Getting the sentences and characters 
train_sentences = train_df["text"].tolist()
val_sentences = val_df["text"].tolist()

# Make function to split sentences into characters
def split_chars(text):
  return " ".join(list(text))

# Split sequence-level data splits into character-level data splits
train_chars = [split_chars(sentence) for sentence in train_sentences]
val_chars = [split_chars(sentence) for sentence in val_sentences]


In [25]:
# Creatinga a text vectorizaiton layer (68k vocab size from the paper itself)
from tensorflow.keras.layers import TextVectorization 

text_vectorizer = TextVectorization(max_tokens= 68000 , 
                                    output_sequence_length = 56)

# Adapt our text vectorizer to training sentences

text_vectorizer.adapt(train_sentences)

In [26]:
# Getting the vocabulary of the vectorizer 
text_vocab = text_vectorizer.get_vocabulary()
len(text_vocab)

64841

In [27]:
# Getting the dict mapping word --> index 
word_index_text = dict(zip(text_vocab , range(len(text_vocab))))
#word_index_char = dict(zip(char_vocab , range(len(char_vocab))))

In [28]:
# Vectorizer for the character level embedding

# Get all keyboard characters for char-level embedding
import string
alphabet = string.ascii_lowercase + string.digits + string.punctuation

# Create char-level token vectorizer instance
NUM_CHAR_TOKENS = len(alphabet) + 2 # num characters in alphabet + space + OOV token
char_vectorizer = TextVectorization(max_tokens=NUM_CHAR_TOKENS,  
                                    output_sequence_length=290,
                                    standardize="lower_and_strip_punctuation",
                                    name="char_vectorizer")

# Adapt character vectorizer to training characters
char_vectorizer.adapt(train_chars)


In [21]:
char_vocab = char_vectorizer.get_vocabulary()

In [29]:
# Creating a function that will give us a embedding matrix 
def get_glove_embedding_matrix(num_tokens , embedding_dim , word_index):

  # Defining the hits and misses here 
  hits , misses = 0 , 0

  # Prepare the embedding matrix 
  embedding_matrix = np.zeros((num_tokens , embedding_dim ))
  for word , i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector 
      hits += 1 
    else:
      misses += 1 

  return embedding_matrix , hits , misses

In [22]:
# Using the above function to get the embedding matrix 

num_tokens_text = len(text_vocab) + 2 
#num_tokens_char = len(char_vocab) + 2 
embedding_dim = 100

sentence_embedding_matrix , hits_ , misses_ = get_glove_embedding_matrix(num_tokens_text , embedding_dim, word_index_text)
#char_embedding_matrix , hits , misses = get_glove_embedding_matrix(num_tokens_char , embedding_dim , word_index_char)


print(f'Hits: {hits_} and Misses: {misses_} for the sentence embedding matrix')
#print(f'Hits: {hits_} and Misses: {misses} for the character level embedding matrix')

Hits: 29730 and Misses: 35111 for the sentence embedding matrix


In [30]:
# Adding the embedding matrix to our Embedding layer (Sentence and characters)
from tensorflow.keras.layers import Embedding

sen_embedding_layer = Embedding(num_tokens_text , 
                            embedding_dim , 
                            embeddings_initializer = tf.keras.initializers.Constant(sentence_embedding_matrix) , 
                            trainable = False )

# char_embedding_layer = Embedding(num_tokens_char , 
#                                  embedding_dim , 
#                                  embeddings_initializer = tf.keras.initializers.Constant(char_embedding_matrix) , 
#                                  trainable = False)


Before making the datasets, we gotta convert our string's into numerical values with the help of the `vectorizer` layers we have created for our both sentence and characters.

In [32]:
# Creating the datasets for our both sentences and chars  

train_sen_vectors = text_vectorizer(np.array([[sen] for sen in train_sentences])).numpy()
val_sen_vectors = text_vectorizer(np.array([[sen] for sen in val_sentences])).numpy()

# train_char_vectors = char_vectorizer(np.array([[char] for char in train_chars])).numpy()
# val_char_vectors = char_vectorizer(np.array([[char] for char in val_chars])).numpy()


# # Converting the above into tf.data datasets 
# train_char_token_data = tf.data.Dataset.from_tensor_slices((train_sen_vectors , train_char_vectors))
# val_char_token_data = tf.data.Dataset.from_tensor_slices((val_sen_vectors , val_char_vectors))

# train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels_encoded)
# val_label_ds = tf.data.Dataset.from_tensor_slices(val_labels_encoded)

# # Zipping the datasets
# train_char_token_ds = tf.data.Dataset.zip((train_char_token_data , train_label_ds))
# val_char_token_ds = tf.data.Dataset.zip((val_char_token_data , val_label_ds))

# Training and validation dataset 
train_ds = tf.data.Dataset.from_tensor_slices((train_sen_vectors , train_labels_encoded))
val_ds = tf.data.Dataset.from_tensor_slices((val_sen_vectors , val_labels_encoded))


# Applying the batch size and prefetching (performance optimization )
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(tf.data.AUTOTUNE)


train_ds,  val_ds

(<PrefetchDataset shapes: ((None, 56), (None,)), types: (tf.int64, tf.int64)>,
 <PrefetchDataset shapes: ((None, 56), (None,)), types: (tf.int64, tf.int64)>)

Perfect! Now we're gonna build a model that will use glove embeddings as the core.

In [67]:
# Set the token inputs 
token_input = layers.Input(shape = () , dtype = tf.int64 , name = 'token_input_layer')
# token_glove_embeddings = 
token_output = sen_embedding_layer(token_input)
#tok_dim = layers.Lambda(lambda x: tf.expand_dims(x , axis = 1))(token_glove_embeddings)
#token_conv = layers.Conv1D(128 , 5 , activation = 'relu' )(tok_dim)
#token_output = layers.MaxPooling1D(5 , padding = 'same')(token_conv)
#token_output = layers.Dense(128 , activation= 'relu')(token_glove_embeddings)
token_model = tf.keras.Model(inputs = token_input , 
                             outputs = token_output)

# Set the char input model
char_input = layers.Input(shape = () , dtype= tf.int64 , name = 'char_input_layer')
#char_glove_embeddings = char_embedding_layer(char_input)
char_output =  char_embedding_layer(char_input)
# char_dim = layers.Lambda(lambda x: tf.expand_dims(x , axis = 1))(char_glove_embeddings)
# char_conv = layers.Conv1D(128 , 5 , activation= 'relu')(char_dim)
# char_output = layers.MaxPooling1D(5 , padding= 'same')(char_conv)
#char_output = layers.Dense(128 , activation= 'relu')(char_glove_embeddings)

char_model = tf.keras.Model(char_input , char_output)

# Concatenate token and char inputs (hybrid token embedding)
token_char_concat = layers.Concatenate(name = 'token_char_hybrid')([token_model.output , char_model.output])

# Create the output layers 
dim = layers.Lambda(lambda x: tf.expand_dims(x , axis = 1))(token_char_concat)
conv = layers.Conv1D(128 , 5 , activation = 'relu' , padding = "same")(dim)
pool = layers.MaxPooling1D(2 , padding = 'same')(conv)
global_maxpool = layers.GlobalMaxPooling1D()(pool)
x = layers.Dense(128 , activation= 'relu')(global_maxpool)
x = layers.Dropout(0.5)(x)

output_layer = layers.Dense(len(class_names) , activation = 'softmax')(x)

# Packing into a model 
glove_model = tf.keras.Model(inputs = [token_model.input , char_model.input] , 
                             outputs = output_layer , 
                             name = 'Glove_embedding_model') 

glove_model.summary()


Model: "Glove_embedding_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token_input_layer (InputLayer)  [(None,)]            0                                            
__________________________________________________________________________________________________
char_input_layer (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         multiple             6484300     token_input_layer[0][0]          
__________________________________________________________________________________________________
embedding_2 (Embedding)         multiple             3000        char_input_layer[0][0]           
______________________________________________________________________________

In [42]:
train_sen_vectors[0].shape

(56,)

In [49]:
# Sample 
input = layers.Input(shape = (None,) , dtype = 'int64')
glove_emb = sen_embedding_layer(input)
#sample_emb = embedding_layer(sample_tokens)
x = layers.Conv1D(128 , 5 , activation= 'relu' , padding = 'same')(glove_emb)
x = layers.MaxPooling1D(5, padding = 'same')(x)
x = layers.Conv1D(128, 5, activation="relu" , padding = 'same')(x)
x = layers.MaxPooling1D(5 , padding ='same')(x)
x = layers.Conv1D(128, 5, activation="relu" , padding = 'same')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(class_names) , activation= 'softmax')(x)

glove_model = tf.keras.Model(input , output)
glove_model.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        multiple                  6484300   
_________________________________________________________________
conv1d_17 (Conv1D)           (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, None, 128)         8204

Now we gotta convert our list of string data to Numpy arrays of integer indicees. Th

In [50]:
# Compiling and fitting the model
glove_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy() , 
                     optimizer = tf.keras.optimizers.Adam(), 
                     metrics = ['accuracy'])

glove_model.fit(train_ds,
                 epochs = 3 , 
                 validation_data = val_ds)

Epoch 1/3
5627/5627 [==============================] - 22s 4ms/step - loss: 0.6537 - accuracy: 0.7588 - val_loss: 0.5387 - val_accuracy: 0.8047
Epoch 2/3
5627/5627 [==============================] - 21s 4ms/step - loss: 0.5301 - accuracy: 0.8084 - val_loss: 0.5207 - val_accuracy: 0.8121
Epoch 3/3
5627/5627 [==============================] - 21s 4ms/step - loss: 0.4854 - accuracy: 0.8245 - val_loss: 0.5542 - val_accuracy: 0.8028


### 2. PubMed Bert 


In [51]:
# Download by uncommenting the below command
!pip install tensorflow_text 

     |████████████████████████████████| 4.4 MB 14.5 MB/s 


In [62]:
# Loading in the both encoder and the preprocessing models 
import tensorflow_text as text
import tensorflow_hub as hub

# preprocess_bert = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
# bert = hub.load('')

preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3' ,
                                     trainable = False , name = 'pubmed_bert_preprocessor')

bert_layer = hub.KerasLayer('https://tfhub.dev/google/experts/bert/pubmed/2' ,
                            trainable = False , 
                            name = 'bert_model_layer')

In [92]:
# Creating a model out of it 
input = layers.Input(shape = [] , dtype = tf.string , name = 'input_sentences')
bert_inputs = preprocessing_layer(input)
bert_embedding =bert_layer(bert_inputs)
print(f'bert embedding shape: {bert_embedding}')
##x = layers.Lambda(lambda x: tf.expand_dims(x, axis = 1))(bert_embedding['pooled_output'])
x = layers.Dense(128 , activation = 'relu')(bert_embedding['pooled_output'])
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(class_names) , activation= 'softmax')(x)

# Packing into a model
pubmed_bert_model = tf.keras.Model(input , output)
pubmed_bert_model.summary()

bert embedding shape: {'sequence_output': <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, 'default': <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert_model_layer')>, 'pooled_output': <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert_model_layer')>, 'encoder_outputs': [<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'bert_model_layer')>, <KerasTens

In [93]:
pubmed_bert_model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy() , 
                          optimizer = tf.keras.optimizers.Adam(), 
                          metrics =['accuracy'])

pubmed_bert_model.fit(train_sen_ds ,
                      steps_per_epoch = int(0.1 * len(train_sen_ds)),
                      epochs = 3 , 
                      validation_data = val_sen_ds , 
                      validation_steps = int(0.1 * len(val_sen_ds)))

Epoch 1/3
562/562 [==============================] - 137s 241ms/step - loss: 0.6510 - accuracy: 0.7732 - val_loss: 0.4593 - val_accuracy: 0.8378
Epoch 2/3
562/562 [==============================] - 135s 240ms/step - loss: 0.5194 - accuracy: 0.8213 - val_loss: 0.4358 - val_accuracy: 0.8424
Epoch 3/3
562/562 [==============================] - 135s 240ms/step - loss: 0.4991 - accuracy: 0.8273 - val_loss: 0.4179 - val_accuracy: 0.8547


In [87]:

train_sen_ds = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels_encoded))
train_sen_ds = train_sen_ds.batch(32).prefetch(tf.data.AUTOTUNE)

val_sen_ds = tf.data.Dataset.from_tensor_slices((val_sentences, val_labels_encoded))
val_sen_ds = val_sen_ds.batch(32).prefetch(tf.data.AUTOTUNE)

### 3 : Boom 

In [99]:
train_df[['line_number' , 'total_lines']].apply(lambda x: x[0].astype('int')  + x[1].astype('int'))

line_number     1
total_lines    22
dtype: int64

AttributeError: ignored

In [102]:
for i , col in enumerate(train_df[['line_number' , 'total_lines']]):
  print(i , col)

0 line_number
1 total_lines
